In [ ]:
#필요 라이브러리 임포트
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

# interactive graph
import plotly.graph_objects as go
import cufflinks as cf 
import plotly.express as px
cf.go_offline(connected=True)

import seaborn as sns

import platform
path = 'c:/Windows/Fonts/malgun.ttf'
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~~')

In [ ]:
titanic_train = pd.read_csv('data/train.csv')
titanic_test = pd.read_csv('data/test.csv')

### 1. 데이터 탐색

In [ ]:
print(titanic_train.shape)
titanic_train.head(3)

In [ ]:
titanic_train.info()

- 12개의 column을 가지고 있고 전체 891개의 데이터

In [ ]:
fig = px.bar(titanic_train,title = 'feature 별 null 값 갯수', x=titanic_train.isnull().sum().sort_values(ascending = False).index, y=titanic_train.isnull().sum().sort_values(ascending = False), text = titanic_train.isnull().sum().sort_values(ascending = False),height= 500)
fig.show()

-  **null 값의 개수** : Cabin은 약 700개, Age는 180개의 null값을 가지고 있다.

In [ ]:
fig = px.bar(titanic_train,title = '생존자 비율 확인', x=[0,1], y=titanic_train['Survived'].value_counts() / len(titanic_train), text = (titanic_train['Survived'].value_counts() / len(titanic_train)).round(2), color=titanic_train['Survived'].value_counts(),color_continuous_scale=['green','red'],width=600, height= 500)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data = titanic_train.groupby(['Sex','Survived'])['Survived'].count()
labels = ["Survived = 0", "Survived = 1"]

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=[data[2], data[3]], name="male", marker_colors = ["red","green"]),1, 1)
fig.add_trace(go.Pie(labels=labels, values=[data[0], data[1]], name="female"),1, 2)

fig.update_traces(hole=.4)

fig.update_layout(
    title_text="성별에 따른 생존자 비율",
    annotations=[dict(text='Male', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='Female', x=0.83, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
# class 별 생존자 확인
sns.catplot(x='Survived', col='Embarked', kind='count', data=titanic_train)

In [ ]:
# 나이, 성별 별 생존자 확인

In [ ]:
sns.distplot(titanic_train['Fare'])

### 2. 데이터 전처리

In [ ]:
def fare(data):
    if data['Age'] == 0:
        if data.Embarked == 'C':
            if data.Pclass == 1:
                return 36.5
            elif data.Pclass == 2:
                return 25
            else:
                return 20
        elif data.Embarked == 'S':
            if data.Pclass == 1:
                return 37
            elif data.Pclass == 2:
                return 30
            else:
                return 25
        else:
            # Q에 대한 인원이 많지 않아 하나로 통합
            return 27
    else:
        return data['Age']
    
def func(age):
    if age <= 2: return 1
    elif age <= 5: return 2
    elif age <= 10: return 3
    elif age <= 18: return 4
    elif age <= 25: return 5
    elif age <= 40: return 6
    elif age <= 60: return 7
    else : return 8

# 0.8339
def data_processing(data):
    # index 설정
    data = data.set_index('PassengerId')
    
    # column 제거
    data.drop(['Name', 'Ticket'], axis = 1, inplace = True)
    
    # Embarked, Cabin null 값 변경
    data['Embarked'].fillna('N',inplace=True)
    data['Cabin'].fillna('N',inplace=True)
    
    # Age의 nan 값 변경 / Embarked, Pclass 별로 중앙값으로 대체
    data['Age'].fillna(0, inplace=True)
    data['new_Age'] = data[['Embarked', 'Pclass', 'Age']].apply(fare, axis = 1)
    data.drop(['Age'], axis = 1, inplace = True)
    
    data['Age'] = data.new_Age.apply(func)
    data.drop(['new_Age'], axis = 1, inplace = True)
    
    data['Cabin'] = data.Cabin.apply(lambda x: x[0])
    data = pd.concat([data, pd.get_dummies(data[['Embarked', 'Sex','Cabin']])], axis = 1)
    data.drop(['Embarked', 'Sex','Cabin'], axis = 1, inplace = True)
    
    # data.drop(['Pclass', 'Sex', 'Embarked', 'Age'], axis = 1, inplace = True)
    
    # data['Sex'] = LabelEncoder().fit(data['Sex']).transform(data['Sex'])
    #data['Embarked'] = LabelEncoder().fit(data['Embarked']).transform(data['Embarked'])
    
    # data['Age'] = LabelEncoder().fit(data['Age']).transform(data['Age'])
    
    # test set의 Fare null 값 대체
    data.Fare.fillna(data['Fare'].median(), inplace = True)
    
    # minmax_scale 적용
    data['Fare'] = minmax_scale(data.Fare)
    data['Age'] = minmax_scale(data.Age)
    #data['SibSp'] = minmax_scale(data.SibSp)
    #data['Parch'] = minmax_scale(data.Parch)
    
    return data

In [ ]:
# 0.8328
def data_processing(data):
    # index 설정
    data = data.set_index('PassengerId')
    
    # column 제거
    data.drop(['Name', 'Ticket','Cabin'], axis = 1, inplace = True)
    
    # Embarked, Cabin null 값 변경
    data['Embarked'].fillna('N',inplace=True)
    # data['Cabin'].fillna('N',inplace=True)
    
    # Age의 nan 값 변경 / Embarked, Pclass 별로 중앙값으로 대체
    data['Age'].fillna(0, inplace=True)
    data['new_Age'] = data[['Embarked', 'Pclass', 'Age']].apply(fare, axis = 1)
    data.drop(['Age'], axis = 1, inplace = True)
    
    data['Age'] = data.new_Age.apply(func)
    data.drop(['new_Age'], axis = 1, inplace = True)
    
    # data['Cabin'] = data.Cabin.apply(lambda x: x[0])
    data = pd.concat([data, pd.get_dummies(data[['Embarked', 'Sex']])], axis = 1)
    data.drop(['Embarked', 'Sex'], axis = 1, inplace = True)
    
    # data.drop(['Pclass', 'Sex', 'Embarked', 'Age'], axis = 1, inplace = True)
    
    # data['Sex'] = LabelEncoder().fit(data['Sex']).transform(data['Sex'])
    #data['Embarked'] = LabelEncoder().fit(data['Embarked']).transform(data['Embarked'])
    
    # data['Age'] = LabelEncoder().fit(data['Age']).transform(data['Age'])
    
    # test set의 Fare null 값 대체
    data.Fare.fillna(data['Fare'].median(), inplace = True)
    
    # minmax_scale 적용
    data['Fare'] = minmax_scale(data.Fare)
    data['Age'] = minmax_scale(data.Age)
    #data['SibSp'] = minmax_scale(data.SibSp)
    #data['Parch'] = minmax_scale(data.Parch)
    
    return data

In [ ]:
# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Sex','Embarked','Cabin']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [ ]:
test = titanic_train.copy()
df = titanic_test.copy()

In [ ]:
test = transform_features(test)

In [ ]:
test.head()

In [ ]:
data = data_processing(test)
data.head()

### 3. 데이터 분석

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

params = { 'max_depth':3,
           'eta': 0.1,
           'objective':'binary:logistic',
           'eval_metric':'logloss',
           'early_stoppings':100
        }
num_rounds = 400
evals = [(X_test, y_test)]

# 0.8328
def data_analysis(data):
    X_train, X_test, y_train, y_test=train_test_split(data.iloc[:,1:], data.iloc[:,0],random_state=10)
 
    dt_clf = DecisionTreeClassifier(random_state=0)
    rf_clf = RandomForestClassifier(random_state=0)
    lr_clf = LogisticRegression()
    knn_clf = KNeighborsClassifier(n_neighbors=10)
    gb_clf = GradientBoostingClassifier(random_state=0)
    
    xgb_clf = XGBClassifier(n_estimators=300, learning_rate=0.01, max_depth=7)
    
    xgb_clf.fit(X_train , y_train, eval_metric="logloss",  verbose=True)
    pred = xgb_clf.predict(X_test)
    print('{0} 정확도: {1:.4f}'.format(xgb_clf.__class__.__name__, accuracy_score(y_test , pred)))
    
    # voting : 0.87 / rf, xgb
    vo_clf = VotingClassifier( estimators=[# ('dt',dt_clf),
                                           # ('rf',rf_clf),
                                           # ('lr',lr_clf),
                                           # ('knn',knn_clf),
                                           ('gb',gb_clf),
                                           ('xgb',xgb_clf)]
                              ,voting='soft')
    vo_clf.fit(X_train , y_train)
    pred = vo_clf.predict(X_test)
    print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

    # 개별 모델의 학습/예측/평가.
    classifiers = [dt_clf, rf_clf, lr_clf, knn_clf, gb_clf]
    for classifier in classifiers:
        classifier.fit(X_train , y_train)
        pred = classifier.predict(X_test)
        class_name= classifier.__class__.__name__
        print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))
    
    # result = rf_pred
    print()
    print()
    # exec_kfold 호출
    classifiers = [gb_clf, rf_clf, xgb_clf, vo_clf]
    for classifier in classifiers:
        print(classifier.__class__.__name__)
        exec_kfold(classifier)
        print()
        print()

In [ ]:
def data_analysis(data):
    X_train, X_test, y_train, y_test=train_test_split(data.iloc[:,1:], data.iloc[:,0],random_state=10)
 
    dt_clf = DecisionTreeClassifier(random_state=0)
    rf_clf = RandomForestClassifier(random_state=0)
    lr_clf = LogisticRegression()
    knn_clf = KNeighborsClassifier(n_neighbors=21)
    gb_clf = GradientBoostingClassifier(random_state=0)
    
    xgb_clf = XGBClassifier(n_estimators=300, learning_rate=0.01, max_depth=7)
    
    xgb_clf.fit(X_train , y_train, eval_metric="logloss",  verbose=True)
    pred = xgb_clf.predict(X_test)
    print('{0} 정확도: {1:.4f}'.format(xgb_clf.__class__.__name__, accuracy_score(y_test , pred)))
    
    # voting : 0.87 / rf, xgb
    vo_clf = VotingClassifier( estimators=[# ('dt',dt_clf),
                                           ('rf',rf_clf),
                                           # ('lr',lr_clf),
                                           # ('knn',knn_clf),
                                           ('gb',gb_clf),
                                           ('xgb',xgb_clf)]
                              ,voting='soft')
    vo_clf.fit(X_train , y_train)
    pred = vo_clf.predict(X_test)
    print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

    # 개별 모델의 학습/예측/평가.
    classifiers = [dt_clf, rf_clf, lr_clf, knn_clf, gb_clf]
    for classifier in classifiers:
        classifier.fit(X_train , y_train)
        pred = classifier.predict(X_test)
        class_name= classifier.__class__.__name__
        print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))
    
    # result = rf_pred
    print()
    print()
    # exec_kfold 호출
    classifiers = [gb_clf, rf_clf, xgb_clf, vo_clf]
    for classifier in classifiers:
        print(classifier.__class__.__name__)
        exec_kfold(data, classifier)
        print()
        print()

In [ ]:
data_analysis(test)

In [ ]:
def exec_kfold(data, clf, folds=7):
    # 폴드 세트를 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위한  리스트 객체 생성.
    kfold = KFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행. 
    for iter_count , (train_index, test_index) in enumerate(kfold.split(data.iloc[:,1:])):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = data.iloc[:,1:].values[train_index], data.iloc[:,1:].values[test_index]
        y_train, y_test = data.iloc[:,0].values[train_index], data.iloc[:,0].values[test_index]
        
        # Classifier 학습, 예측, 정확도 계산 
        clf.fit(X_train, y_train) 
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))     
    
    # 5개 fold에서의 평균 정확도 계산. 
    mean_score = np.mean(scores)
    print("평균 정확도: {0:.4f}".format(mean_score)) 

In [1]:
#필요 라이브러리 임포트
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

# interactive graph
import plotly.graph_objects as go
import cufflinks as cf 
import plotly.express as px
cf.go_offline(connected=True)

import seaborn as sns

import platform
path = 'c:/Windows/Fonts/malgun.ttf'
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~~')

In [2]:
# 0.8328
def data_processing(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    
    return df

In [3]:
titanic_df = pd.read_csv('data/train.csv')
titanic_test = pd.read_csv('data/test.csv')

In [5]:
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived',axis=1)

X_titanic_df = data_processing(X_titanic_df)

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

params = { 'max_depth':3,
           'eta': 0.1,
           'objective':'binary:logistic',
           'eval_metric':'logloss',
           'early_stoppings':100
        }
num_rounds = 400


# 0.8328
def data_analysis(data):
    X_train, X_test, y_train, y_test=train_test_split(data.iloc[:,1:], data.iloc[:,0],random_state=10)
    evals = [(X_test, y_test)]
 
    dt_clf = DecisionTreeClassifier(random_state=0)
    rf_clf = RandomForestClassifier(random_state=0)
    lr_clf = LogisticRegression()
    knn_clf = KNeighborsClassifier(n_neighbors=10)
    gb_clf = GradientBoostingClassifier(random_state=0)
    
    xgb_clf = XGBClassifier(n_estimators=300, learning_rate=0.01, max_depth=7)
    
    xgb_clf.fit(X_train , y_train, eval_metric="logloss",  verbose=True)
    pred = xgb_clf.predict(X_test)
    print('{0} 정확도: {1:.4f}'.format(xgb_clf.__class__.__name__, accuracy_score(y_test , pred)))
    
    # voting : 0.87 / rf, xgb
    vo_clf = VotingClassifier( estimators=[('dt',dt_clf),
                                           # ('rf',rf_clf),
                                           # ('lr',lr_clf),
                                           # ('knn',knn_clf),
                                           ('gb',gb_clf),
                                           ('xgb',xgb_clf)]
                              ,voting='soft')
    vo_clf.fit(X_train , y_train)
    pred = vo_clf.predict(X_test)
    print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

    # 개별 모델의 학습/예측/평가.
    classifiers = [dt_clf, rf_clf, lr_clf, knn_clf, gb_clf]
    for classifier in classifiers:
        classifier.fit(X_train , y_train)
        pred = classifier.predict(X_test)
        class_name= classifier.__class__.__name__
        print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))
    
    # result = rf_pred
    print()
    print()
    # exec_kfold 호출
    classifiers = [gb_clf, rf_clf, xgb_clf, vo_clf]
    for classifier in classifiers:
        print(classifier.__class__.__name__)
        exec_kfold(data, classifier)
        print()
        print()

In [10]:
def exec_kfold(data, clf, folds=7):
    # 폴드 세트를 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위한  리스트 객체 생성.
    kfold = KFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행. 
    for iter_count , (train_index, test_index) in enumerate(kfold.split(data.iloc[:,1:])):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = data.iloc[:,1:].values[train_index], data.iloc[:,1:].values[test_index]
        y_train, y_test = data.iloc[:,0].values[train_index], data.iloc[:,0].values[test_index]
        
        # Classifier 학습, 예측, 정확도 계산 
        clf.fit(X_train, y_train) 
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))     
    
    # 5개 fold에서의 평균 정확도 계산. 
    mean_score = np.mean(scores)
    print("평균 정확도: {0:.4f}".format(mean_score)) 

In [11]:
data_analysis(X_titanic_df)

XGBClassifier 정확도: 0.9462
Voting 분류기 정확도: 0.9507
DecisionTreeClassifier 정확도: 0.9507
RandomForestClassifier 정확도: 0.9372
LogisticRegression 정확도: 0.8117
KNeighborsClassifier 정확도: 0.8475
GradientBoostingClassifier 정확도: 0.9552


GradientBoostingClassifier
교차 검증 0 정확도: 0.9688
교차 검증 1 정확도: 0.9297
교차 검증 2 정확도: 0.9685
교차 검증 3 정확도: 0.9528
교차 검증 4 정확도: 0.9528
교차 검증 5 정확도: 0.9528
교차 검증 6 정확도: 0.9370
평균 정확도: 0.9517


RandomForestClassifier
교차 검증 0 정확도: 0.9375
교차 검증 1 정확도: 0.9297
교차 검증 2 정확도: 0.9528
교차 검증 3 정확도: 0.8898
교차 검증 4 정확도: 0.9291
교차 검증 5 정확도: 0.9449
교차 검증 6 정확도: 0.9055
평균 정확도: 0.9270


XGBClassifier
교차 검증 0 정확도: 0.9531
교차 검증 1 정확도: 0.9141
교차 검증 2 정확도: 0.9606
교차 검증 3 정확도: 0.9606
교차 검증 4 정확도: 0.9370
교차 검증 5 정확도: 0.9449
교차 검증 6 정확도: 0.9134
평균 정확도: 0.9405


VotingClassifier
교차 검증 0 정확도: 0.9453
교차 검증 1 정확도: 0.9297
교차 검증 2 정확도: 0.9528
교차 검증 3 정확도: 0.9528
교차 검증 4 정확도: 0.9449
교차 검증 5 정확도: 0.9370
교차 검증 6 정확도: 0.9213
평균 정확도: 0.9405


